# Some experiments on PGN data

### Imports

In [34]:
import os
import time
import chess
import chess.pgn

### Parsing 50k games

In [19]:
start = time.time()

pgn = open('../data/fics_202011_notime_50k.pgn')

def doNothing():
    return

while chess.pgn.read_game(pgn):
    doNothing()

end = time.time()
print(f'Time elapsed: {end - start}')

Time elapsed: 274.8620150089264


### Calculating relative value of pieces for a given colour

In [30]:
# [pawn, knight, bishop, rook, queen], see https://en.wikipedia.org/wiki/Chess_piece_relative_value
piece_values = [1, 3, 3, 5, 9]

# Given chess.Board and chess.Color
# Returns sum of piece values for that color
def get_piece_value(board, color):
    piece_value_sum = 0
    for i in range(0, 5):
        piece_value_sum += piece_values[i] * len(board.pieces(i+1, color))
    return piece_value_sum

In [32]:
pgn = open('../data/fics_202011_notime_50k.pgn')

game = chess.pgn.read_game(pgn)
board = game.board()
get_piece_value(board, chess.BLACK)

39

### Game metadata

In [51]:
game.headers

Headers(Event='FICS rated blitz game', Site='FICS freechess.org', Date='2020.11.30', Round='?', White='stevharr', Black='nimzoii', Result='1-0', BlackClock='0:05:00.000', BlackElo='1337', BlackRD='0.0', ECO='B32', FICSGamesDBGameNo='474470039', PlyCount='117', Time='23:49:00', TimeControl='300+0', WhiteClock='0:05:00.000', WhiteElo='1677', WhiteRD='0.0')

In [ ]:
# Given chess.pgn.Game and chess.Color
# Return -1 if draw, 1 if color won, 0 if color lost.
def get_game_result(game, color):
    if '1/2' in game.headers['Result']:
        return -1
    elif color == chess.WHITE:
        return game.headers['Result'][0]
    else:
        return game.headers['Result'][2]

In [53]:
white_elo = game.headers['WhiteElo']
black_elo = game.headers['BlackElo']
    
print(f'White Elo: {white_elo}, Black Elo: {black_elo}')
print(f'Result for white: {get_game_result(game, chess.WHITE)}')

White Elo: 1677, Black Elo: 1337
Result for white: 1
